# PyConAr 2018 - Taller API Georef

Requerimientos:
 - Python 3
 - jupyter
 - pandas
 - requests

In [1]:
import json
import os
import pandas as pd
import requests

### Funciones de utilidad

Se definen dos funciones de utilidad:

La función `load_csv` carga un archivo CSV y toma una muestra aleatoria de 100 elementos (por defecto). El muestreado se realiza para poder agilizar el taller y no tener que manejar cantidades de datos demasiado grandes.

In [2]:
def load_csv(name, length=100, random_state=1):
    # Cargar archivo CSV de la carpeta 'datos', eliminando duplicados
    dataframe = pd.read_csv(os.path.join('datos', name)).drop_duplicates()
    
    # Tomar una muestra 'aleatoria' de N elementos
    dataframe = dataframe.sample(n=min(len(dataframe.index), length), random_state=random_state)
    
    # Asegurarse de que los índices sean secuenciales
    return dataframe.reset_index(drop=True)

La función `expand_geojson_column` toma un DataFrame con una columna `geojson`, y devuelve un nuevo DataFrame con dos columnas: `longitud` y `latitud`, con los valores correspondientes de la columna original. El contenido de cada celda de la columna `geojson` debe ser un valor de tipo `Point`.

In [3]:
def expand_geojson_column(data):
    # Toma un DataFrame con una columna 'geojson' y retorna un DataFrame con 'lat' y 'lon'
    def geojson_lat_lon(value):
        geojson = json.loads(value)
        return geojson['coordinates']
    
    col = data[['geojson']].applymap(geojson_lat_lon)
    return col['geojson'].apply(pd.Series).rename(columns={0: 'longitud', 1: 'latitud'})

Como ejemplo, se construye un DataFrame `geojson_df`, y se lo une con el resultado de expandir su columna `geojson` a `longitud` y `latitud`:

In [4]:
geojson_df = pd.DataFrame([
    '{"type":"Point","coordinates":[-65.5497560941612,-22.3231238419683]}',
    '{"type":"Point","coordinates":[-66.1589398249571,-22.3505830769888]}'
], columns=['geojson'])

geojson_df.join(expand_geojson_column(geojson_df))

,geojson,longitud,latitud
0,"{""type"":""Point"",""coordinates"":[-65.54975609416...",-65.549756,-22.323124
1,"{""type"":""Point"",""coordinates"":[-66.15893982495...",-66.158940,-22.350583


### Funciones de normalización

Se definen dos funciones de normalización de datos:

<Explicar el funcionamiento de la funcion `georef_normalize_entities`>

In [5]:
def georef_normalize_entities(data, entity, fields, params=None):
    if not params:
        params = {}
    queries = []

    for i in data.index:
        query = {
            'max': 1,
            'aplanar': True,
            **params
        }

        for key, value in fields.items():
            query[key] = data.loc[i, value]

        queries.append(query)

    body = {
        entity: queries
    }

    url = 'https://apis.datos.gob.ar/georef/api/{}'.format(entity)

    resp = requests.post(url, json=body)
    resp.raise_for_status()
    results = resp.json()['resultados']

    return pd.DataFrame(result[entity][0] if result[entity] else {} for result in results)

<Explicar el funcionamiento de la funcion `georef_reverse_geocode`>

In [6]:
def georef_reverse_geocode(data, fields, params=None):
    if not params:
        params = {}
    queries = []

    for i in data.index:
        query = {
            'aplanar': True,
            **params
        }

        for key, value in fields.items():
            query[key] = data.loc[i, value]

        queries.append(query)

    body = {
        'ubicaciones': queries
    }

    resp = requests.post('https://apis.datos.gob.ar/georef/api/ubicacion', json=body)
    resp.raise_for_status()
    results = resp.json()['resultados']

    return pd.DataFrame(result['ubicacion'] for result in results)